In [1]:
using PowerModels
using Ipopt
using JuMP

In [2]:
using Memento

# Create our module level logger (this will get precompiled)
const _LOGGER = Memento.getlogger(@__MODULE__)

# Register the module level logger at runtime so that folks can access the logger via `getlogger(PowerModels)`
# NOTE: If this line is not included then the precompiled `PowerModels._LOGGER` won't be registered at runtime.
__init__() = Memento.register(_LOGGER)

__init__ (generic function with 1 method)

In [3]:
import InfrastructureModels
import InfrastructureModels: ids, ref, var, con, sol, nw_ids, nws, optimize_model!, @im_fields
const _IM = InfrastructureModels

InfrastructureModels

In [4]:
import MathOptInterface
const _MOI = MathOptInterface

MathOptInterface

In [5]:
function _create_modifications(pm::AbstractPowerModel,
    vm_lb::Dict{Any,Float64}, vm_ub::Dict{Any,Float64},
    td_lb::Dict{Any,Float64}, td_ub::Dict{Any,Float64})

    println("Abhijeet : _create_modifications")
    modifications = Dict{String,Any}()

    modifications["per_unit"] = true
    modifications["bus"] = Dict{String,Any}()
    modifications["branch"] = Dict{String,Any}()

    for bus in ids(pm, :bus)
        index = string(ref(pm, :bus, bus, "index"))
        modifications["bus"][index] = Dict("vmin" => vm_lb[bus], "vmax" => vm_ub[bus])
    end

    for branch in ids(pm, :branch)
        index = string(ref(pm, :branch, branch, "index"))
        f_bus = ref(pm, :branch, branch, "f_bus")
        t_bus = ref(pm, :branch, branch, "t_bus")
        bp = (f_bus, t_bus)
        modifications["branch"][index] = Dict("angmin" => td_lb[bp], "angmax" => td_ub[bp])
    end

    return modifications
end

_create_modifications (generic function with 1 method)

In [6]:
function _constraint_obj_bound(pm::AbstractPowerModel, bound)
    println("Abhijeet : _constraint_obj_bound")
    model = PowerModels.check_cost_models(pm)
    if model != 2
        Memento.error(_LOGGER, "Only cost models of type 2 is supported at this time, given cost model type $(model)")
    end

    cost_index = PowerModels.calc_max_cost_index(pm.data)
    if cost_index > 3
        Memento.error(_LOGGER, "Only quadratic generator cost models are supported at this time, given cost model of order $(cost_index-1)")
    end

    PowerModels.standardize_cost_terms!(pm.data, order=2)

    from_idx = Dict(arc[1] => arc for arc in ref(pm, :arcs_from_dc))

    JuMP.@constraint(pm.model,
            sum(
                gen["cost"][1]*var(pm, :pg, i)^2 +
                gen["cost"][2]*var(pm, :pg, i) +
                gen["cost"][3]
            for (i,gen) in ref(pm, :gen)) +
            sum(
                dcline["cost"][1]*var(pm,:p_dc, from_idx[i])^2 +
                dcline["cost"][2]*var(pm, :p_dc, from_idx[i])  +
                dcline["cost"][3]
            for (i,dcline) in ref(pm, :dcline))
            <= bound
    )
end

_constraint_obj_bound (generic function with 1 method)

In [7]:
function _check_obbt_options(ub::Float64, rel_gap::Float64, ub_constraint::Bool)
    println("Abhijeet : _check_obbt_options")
    if ub_constraint && isinf(ub)
        Memento.error(_LOGGER, "the option upper_bound_constraint cannot be set to true without specifying an upper bound")
    end

    if !isinf(rel_gap) && isinf(ub)
        Memento.error(_LOGGER, "rel_gap_tol is specified without providing an upper bound")
    end
end

_check_obbt_options (generic function with 1 method)

In [8]:
function _check_variables(pm::AbstractPowerModel)
    println("Abhijeet : _check_variables")
    try
        vm = var(pm, :vm)
    catch err
        (isa(error, KeyError)) && (Memento.error(_LOGGER, "OBBT is not supported for models without explicit voltage magnitude variables"))
    end

    try
        td = var(pm, :td)
    catch err
        (isa(error, KeyError)) && (Memento.error(_LOGGER, "OBBT is not supported for models without explicit voltage angle difference variables"))
    end
end

_check_variables (generic function with 1 method)

In [9]:
function run_obbt_opf!(data::Dict{String,<:Any}, optimizer;
    model_type::Type = QCLSPowerModel,
    max_iter::Int = 100,
    time_limit::Float64 = 3600.0,
    upper_bound::Float64 = Inf,
    upper_bound_constraint::Bool = false,
    rel_gap_tol::Float64 = Inf,
    min_bound_width::Float64 = 1e-2,
    improvement_tol::Float64 = 1e-3,
    precision::Int = 4,
    termination::Symbol = :avg,
    kwargs...)

    println("Abhijeet : run_obbt_opf")
    Memento.info(_LOGGER, "maximum OBBT iterations set to default value of $max_iter")
    Memento.info(_LOGGER, "maximum time limit for OBBT set to default value of $time_limit seconds")

    model_relaxation = instantiate_model(data, model_type, PowerModels.build_opf)
    (_IM.ismultinetwork(model_relaxation)) && (Memento.error(_LOGGER, "OBBT is not supported for multi-networks"))
    (ismulticonductor(model_relaxation)) && (Memento.error(_LOGGER, "OBBT is not supported for multi-conductor networks"))

    # check for model_type compatability with OBBT
    _check_variables(model_relaxation)

    # check for other keyword argument consistencies
    _check_obbt_options(upper_bound, rel_gap_tol, upper_bound_constraint)

    # check termination norm criteria for obbt
    (termination != :avg && termination != :max) && (Memento.error(_LOGGER, "OBBT termination criteria can only be :max or :avg"))

    # pass status
    status_pass = [_MOI.LOCALLY_SOLVED, _MOI.OPTIMAL]

    # compute initial relative gap between relaxation objective and upper_bound
    result_relaxation = optimize_model!(model_relaxation, optimizer=optimizer)
    current_relaxation_objective = result_relaxation["objective"]
    if upper_bound < current_relaxation_objective
        Memento.error(_LOGGER, "the upper bound provided to OBBT is not a valid ACOPF upper bound")
    end
    if !(result_relaxation["termination_status"] in status_pass)
        Memento.warn(_LOGGER, "initial relaxation solve status is $(result_relaxation["termination_status"])")
        if result_relaxation["termination_status"] == :SubOptimal
            Memento.warn(_LOGGER, "continuing with the bound-tightening algorithm")
        end
    end
    current_rel_gap = Inf
    if !isinf(upper_bound)
        current_rel_gap = (upper_bound - current_relaxation_objective)/upper_bound
        Memento.info(_LOGGER, "Initial relaxation gap = $current_rel_gap")
    end


    model_bt = instantiate_model(data, model_type, PowerModels.build_opf)
    (upper_bound_constraint) && (_constraint_obj_bound(model_bt, upper_bound))

    stats = Dict{String,Any}()
    stats["model_type"] = model_type
    stats["initial_relaxation_objective"] = current_relaxation_objective
    stats["initial_rel_gap_from_ub"] = current_rel_gap
    stats["upper_bound"] = upper_bound

    vm = var(model_bt, :vm)
    td = var(model_bt, :td)
    buses = ids(model_bt, :bus)
    buspairs = ids(model_bt, :buspairs)

    vm_lb = Dict{Any,Float64}( [bus => JuMP.lower_bound(vm[bus]) for bus in buses] )
    vm_ub = Dict{Any,Float64}( [bus => JuMP.upper_bound(vm[bus]) for bus in buses] )
    td_lb = Dict{Any,Float64}( [bp => JuMP.lower_bound(td[bp]) for bp in buspairs] )
    td_ub = Dict{Any,Float64}( [bp => JuMP.upper_bound(td[bp]) for bp in buspairs] )

    vm_range_init = sum([vm_ub[bus] - vm_lb[bus] for bus in buses])
    stats["vm_range_init"] = vm_range_init
    stats["avg_vm_range_init"] = vm_range_init/length(buses)

    td_range_init = sum([td_ub[bp] - td_lb[bp] for bp in buspairs])
    stats["td_range_init"] = td_range_init
    stats["avg_td_range_init"] = td_range_init/length(buspairs)

    vm_range_final = 0.0
    td_range_final = 0.0

    total_vm_reduction = Inf
    total_td_reduction = Inf
    max_vm_reduction = Inf
    max_td_reduction = Inf
    avg_vm_reduction = Inf
    avg_td_reduction = Inf

    final_relaxation_objective = NaN

    current_iteration = 0
    time_elapsed = 0.0
    parallel_time_elapsed = 0.0

    check_termination = true
    (termination == :avg) && (check_termination = (avg_vm_reduction > improvement_tol || avg_td_reduction > improvement_tol))
    (termination == :max) && (check_termination = (max_vm_reduction > improvement_tol || max_td_reduction > improvement_tol))

    while check_termination
        iter_start_time = time()
        total_vm_reduction = 0.0
        avg_vm_reduction = 0.0
        max_vm_reduction = 0.0
        total_td_reduction = 0.0
        avg_td_reduction = 0.0
        max_td_reduction = 0.0
        max_vm_iteration_time = 0.0
        max_td_iteration_time = 0.0


        # bound-tightening for the vm variables
        for bus in buses
            (vm_ub[bus] - vm_lb[bus] < min_bound_width) && (continue)

            start_time = time()
            # vm lower bound solve
            lb = NaN
            JuMP.@objective(model_bt.model, Min, vm[bus])
            result_bt = optimize_model!(model_bt, optimizer=optimizer)
            if (result_bt["termination_status"] == _MOI.LOCALLY_SOLVED || result_bt["termination_status"] == _MOI.OPTIMAL)
                nlb = floor(10.0^precision * JuMP.objective_value(model_bt.model))/(10.0^precision)
                (nlb > vm_lb[bus]) && (lb = nlb)
            else
                Memento.warn(_LOGGER, "BT minimization problem for vm[$bus] errored - change tolerances.")
                continue
            end

            #vm upper bound solve
            ub = NaN
            JuMP.@objective(model_bt.model, Max, vm[bus])
            result_bt = optimize_model!(model_bt, optimizer=optimizer)
            if (result_bt["termination_status"] == _MOI.LOCALLY_SOLVED || result_bt["termination_status"] == _MOI.OPTIMAL)
                nub = ceil(10.0^precision * JuMP.objective_value(model_bt.model))/(10.0^precision)
                (nub < vm_ub[bus]) && (ub = nub)
            else
                Memento.warn(_LOGGER, "BT maximization problem for vm[$bus] errored - change tolerances.")
                continue
            end
            end_time = time() - start_time
            max_vm_iteration_time = max(end_time, max_vm_iteration_time)

            # sanity checks
            (lb > ub) && (Memento.warn(_LOGGER, "bt lb > ub - adjust tolerances in optimizer to avoid issue"); continue)
            (!isnan(lb) && lb > vm_ub[bus]) && (lb = vm_lb[bus])
            (!isnan(ub) && ub < vm_lb[bus]) && (ub = vm_ub[bus])
            isnan(lb) && (lb = vm_lb[bus])
            isnan(ub) && (ub = vm_ub[bus])

            # vm bound-reduction computation
            vm_reduction = 0.0
            if (ub - lb >= min_bound_width)
                vm_reduction = (vm_ub[bus] - vm_lb[bus]) - (ub - lb)
                vm_lb[bus] = lb
                vm_ub[bus] = ub
            else
                mean = (ub + lb)/2.0
                if (mean - min_bound_width/2.0 < vm_lb[bus])
                    lb = vm_lb[bus]
                    ub = vm_lb[bus] + min_bound_width
                elseif (mean + min_bound_width/2.0 > vm_ub[bus])
                    ub = vm_ub[bus]
                    lb = vm_ub[bus] - min_bound_width
                else
                    lb = mean - (min_bound_width/2.0)
                    ub = mean + (min_bound_width/2.0)
                end
                vm_reduction = (vm_ub[bus] - vm_lb[bus]) - (ub - lb)
                vm_lb[bus] = lb
                vm_ub[bus] = ub
            end

            total_vm_reduction += (vm_reduction)
            max_vm_reduction = max(vm_reduction, max_vm_reduction)
        end
        avg_vm_reduction = total_vm_reduction/length(buses)

        vm_range_final = sum([vm_ub[bus] - vm_lb[bus] for bus in buses])

        # bound-tightening for the td variables
        for bp in buspairs
            (td_ub[bp] - td_lb[bp] < min_bound_width) && (continue)

            start_time = time()
            # td lower bound solve
            lb = NaN
            JuMP.@objective(model_bt.model, Min, td[bp])
            result_bt = optimize_model!(model_bt, optimizer=optimizer)
            if (result_bt["termination_status"] == _MOI.LOCALLY_SOLVED || result_bt["termination_status"] == _MOI.OPTIMAL)
                nlb = floor(10.0^precision * JuMP.objective_value(model_bt.model))/(10.0^precision)
                (nlb > td_lb[bp]) && (lb = nlb)
            else
                Memento.warn(_LOGGER, "BT minimization problem for td[$bp] errored - change tolerances")
                continue
            end

            # td upper bound solve
            ub = NaN
            JuMP.@objective(model_bt.model, Max, td[bp])
            result_bt = optimize_model!(model_bt, optimizer=optimizer)
            if (result_bt["termination_status"] == _MOI.LOCALLY_SOLVED || result_bt["termination_status"] == _MOI.OPTIMAL)
                nub = ceil(10.0^precision * JuMP.objective_value(model_bt.model))/(10.0^precision)
                (nub < td_ub[bp]) && (ub = nub)
            else
                Memento.warn(_LOGGER, "BT maximization problem for td[$bp] errored - change tolerances.")
                continue
            end
            end_time = time() - start_time
            max_td_iteration_time = max(end_time, max_td_iteration_time)

            # sanity checks
            (lb > ub) && (Memento.warn(_LOGGER, "bt lb > ub - adjust tolerances in optimizer to avoid issue"); continue)
            (!isnan(lb) && lb > td_ub[bp]) && (lb = td_lb[bp])
            (!isnan(ub) && ub < td_lb[bp]) && (ub = td_ub[bp])
            isnan(lb) && (lb = td_lb[bp])
            isnan(ub) && (ub = td_ub[bp])

            # td bound-reduction computation
            td_reduction = 0.0
            if (ub - lb >= min_bound_width)
                td_reduction = (td_ub[bp] - td_lb[bp]) - (ub - lb)
                td_lb[bp] = lb
                td_ub[bp] = ub
            else
                mean = (lb + ub)/2.0
                if (mean - min_bound_width/2.0 < td_lb[bp])
                    lb = td_lb[bp]
                    ub = td_lb[bp] + min_bound_width
                elseif (mean + min_bound_width/2.0 > td_ub[bp])
                    ub = td_ub[bp]
                    lb = td_ub[bp] - min_bound_width
                else
                    lb = mean - (min_bound_width/2.0)
                    ub = mean + (min_bound_width/2.0)
                end
                td_reduction = (td_ub[bp] - td_lb[bp]) - (ub - lb)
                td_lb[bp] = lb
                td_ub[bp] = ub
            end

            total_td_reduction += (td_reduction)
            max_td_reduction = max(td_reduction, max_td_reduction)

        end
        avg_td_reduction = total_td_reduction/length(buspairs)

        td_range_final = sum([td_ub[bp] - td_lb[bp] for bp in buspairs])

        parallel_time_elapsed += max(max_vm_iteration_time, max_td_iteration_time)

        time_elapsed += (time() - iter_start_time)

        # populate the modifications, update the data, and rebuild the bound-tightening model
        modifications = _create_modifications(model_bt, vm_lb, vm_ub, td_lb, td_ub)
        PowerModels.update_data!(data, modifications)
        model_bt = instantiate_model(data, model_type, PowerModels.build_opf)
        (upper_bound_constraint) && (_constraint_obj_bound(model_bt, upper_bound))
        vm = var(model_bt, :vm)
        td = var(model_bt, :td)

        # run the qc relaxation for the updated bounds
        result_relaxation = run_opf(data, model_type::Type, optimizer)

        if result_relaxation["termination_status"] in status_pass
            current_rel_gap = (upper_bound - result_relaxation["objective"])/upper_bound
            final_relaxation_objective = result_relaxation["objective"]
        else
            Memento.warn(_LOGGER, "relaxation solve failed in iteration $(current_iteration+1)")
            Memento.warn(_LOGGER, "using the previous iteration's gap to check relative gap stopping criteria")
        end

        Memento.info(_LOGGER, "iteration $(current_iteration+1), vm range: $vm_range_final, td range: $td_range_final, relaxation obj: $final_relaxation_objective")

        # termination criteria update
        (termination == :avg) && (check_termination = (avg_vm_reduction > improvement_tol || avg_td_reduction > improvement_tol))
        (termination == :max) && (check_termination = (max_vm_reduction > improvement_tol || max_td_reduction > improvement_tol))
        # interation counter update
        current_iteration += 1
        # check all the stopping criteria
        (current_iteration >= max_iter) && (Memento.info(_LOGGER, "maximum iteration limit reached"); break)
        (time_elapsed > time_limit) && (Memento.info(_LOGGER, "maximum time limit reached"); break)
        if (!isinf(rel_gap_tol)) && (current_rel_gap < rel_gap_tol)
            Memento.info(_LOGGER, "relative optimality gap < $rel_gap_tol")
            break
        end

    end

    branches_vad_same_sign_count = 0
    for (key, branch) in data["branch"]
        is_same_sign = (branch["angmax"] >=0 && branch["angmin"] >= 0) || (branch["angmax"] <=0 && branch["angmin"] <= 0)
        (is_same_sign) && (branches_vad_same_sign_count += 1)
    end

    stats["final_relaxation_objective"] = final_relaxation_objective
    stats["final_rel_gap_from_ub"] = isnan(upper_bound) ? Inf : current_rel_gap
    stats["vm_range_final"] = vm_range_final
    stats["avg_vm_range_final"] = vm_range_final/length(buses)

    stats["td_range_final"] = td_range_final
    stats["avg_td_range_final"] = td_range_final/length(buspairs)

    stats["run_time"] = time_elapsed
    stats["iteration_count"] = current_iteration
    stats["sim_parallel_run_time"] = parallel_time_elapsed

    stats["vad_sign_determined"] = branches_vad_same_sign_count

    return data, stats

end


run_obbt_opf! (generic function with 1 method)

In [10]:
function run_obbt_opf!(file::String, optimizer; kwargs...)
    println("Entry to solve Abhijeet : run_obbt_opf")
    data = PowerModels.parse_file(file)
    return run_obbt_opf!(data, optimizer; kwargs...)
end

run_obbt_opf! (generic function with 2 methods)

In [11]:
result_ac = run_ac_opf("matpower/case3.m", with_optimizer(Ipopt.Optimizer))
upper_bound = result_ac["objective"]
println(upper_bound)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[11]:1
└ @ Core In[11]:1


[info | PowerModels]: extending matpower format with constant data: const_int
[info | PowerModels]: extending matpower format with constant data: const_float
[info | PowerModels]: extending matpower format with data: bus_data 3x3
[info | PowerModels]: extending matpower format with data: branch_names 3x3
[info | PowerModels]: extending matpower format with data: areas_cells 2x6
[info | PowerModels]: extending matpower format with data: areas_named_cells 2x6
[info | PowerModels]: extending matpower format with data: areas_named 2x3
[info | PowerModels]: extending matpower format with data: component 1x3
[info | PowerModels]: extending matpower format with data: areas 2x3
[info | PowerModels]: extending matpower format with constant data: const_str
[info | PowerModels]: extending matpower format with data: load_data 2x2
[info | PowerModels]: extending matpower format with data: branch_limit 3x3
[warn | PowerModels]: added zero cost function data for dclines
[info | PowerModels]: extendin

In [12]:
data, stats = run_obbt_opf!("matpower/case3.m", with_optimizer(Ipopt.Optimizer), model_type=QCRMPowerModel);
println(stats["final_relaxation_objective"])
println(stats["final_rel_gap_from_ub"])
println(stats["iteration_count"])

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[12]:1
└ @ Core In[12]:1


Entry to solve Abhijeet : run_obbt_opf
[info | PowerModels]: extending matpower format with constant data: const_int
[info | PowerModels]: extending matpower format with constant data: const_float
[info | PowerModels]: extending matpower format with data: bus_data 3x3
[info | PowerModels]: extending matpower format with data: branch_names 3x3
[info | PowerModels]: extending matpower format with data: areas_cells 2x6
[info | PowerModels]: extending matpower format with data: areas_named_cells 2x6
[info | PowerModels]: extending matpower format with data: areas_named 2x3
[info | PowerModels]: extending matpower format with data: component 1x3
[info | PowerModels]: extending matpower format with data: areas 2x3
[info | PowerModels]: extending matpower format with constant data: const_str
[info | PowerModels]: extending matpower format with data: load_data 2x2
[info | PowerModels]: extending matpower format with data: branch_limit 3x3
[warn | PowerModels]: added zero cost function data for

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      195
Number of nonzeros in Lagrangian Hessian.............:       27

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       75
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       39

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      195
Number of nonzeros in Lagrangian Hessian.............:       27

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       75
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       39

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      195
Number of nonzeros in Lagrangian Hessian.............:       27

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       75
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       39

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

...

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      204
Number of nonzeros in Lagrangian Hessian.............:       27

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       75
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       39

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      204
Number of nonzeros in Lagrangian Hessian.............:       27

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       75
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       39

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      204
Number of nonzeros in Lagrangian Hessian.............:       27

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       75
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       39

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

[warn | InfrastructureModels]: Model already contains optimizer, cannot use optimizer specified in `optimize_model!`
This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      204
Number of nonzeros in Lagrangian Hessian.............:       27

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       75
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with

5900.040000370445
NaN
5


In [14]:
data2, stats2 = run_obbt_opf!("matpower/case3.m", with_optimizer(Ipopt.Optimizer), 
            model_type = QCRMPowerModel,
            upper_bound = upper_bound, 
            upper_bound_constraint = true, 
            rel_gap_tol = 1e-3);

println(stats2["final_rel_gap_from_ub"])
println(stats2["iteration_count"])
println(stats2["vm_range_final"])

Entry to solve Abhijeet : run_obbt_opf
[info | PowerModels]: extending matpower format with constant data: const_int
[info | PowerModels]: extending matpower format with constant data: const_float
[info | PowerModels]: extending matpower format with data: bus_data 3x3
[info | PowerModels]: extending matpower format with data: branch_names 3x3
[info | PowerModels]: extending matpower format with data: areas_cells 2x6
[info | PowerModels]: extending matpower format with data: areas_named_cells 2x6
[info | PowerModels]: extending matpower format with data: areas_named 2x3
[info | PowerModels]: extending matpower format with data: component 1x3
[info | PowerModels]: extending matpower format with data: areas 2x3
[info | PowerModels]: extending matpower format with constant data: const_str
[info | PowerModels]: extending matpower format with data: load_data 2x2
[info | PowerModels]: extending matpower format with data: branch_limit 3x3
[warn | PowerModels]: added zero cost function data for

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[14]:1
└ @ Core In[14]:1


[info | Main]: Initial relaxation gap = 0.021474644657030993
Abhijeet : _constraint_obj_bound
[warn | PowerModels]: Updated generator 3 cost function with order 0 to a function of order 3: [0.0, 0.0, 0.0]
[warn | PowerModels]: Updated dcline 1 cost function with order 0 to a function of order 3: [0.0, 0.0, 0.0]
This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      199
Number of nonzeros in Lagrangian Hessian.............:       29

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality cons

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      199
Number of nonzeros in Lagrangian Hessian.............:       29

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       76
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       40

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      199
Number of nonzeros in Lagrangian Hessian.............:       29

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       76
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       40

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

...

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      208
Number of nonzeros in Lagrangian Hessian.............:       29

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       76
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       40

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      208
Number of nonzeros in Lagrangian Hessian.............:       29

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       76
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       40

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      208
Number of nonzeros in Lagrangian Hessian.............:       29

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       76
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       40

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

[warn | InfrastructureModels]: Model already contains optimizer, cannot use optimizer specified in `optimize_model!`
This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       96
Number of nonzeros in inequality constraint Jacobian.:      208
Number of nonzeros in Lagrangian Hessian.............:       29

Total number of variables............................:       51
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Total number of equality constraints.................:       26
Total number of inequality constraints...............:       76
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with

[info | Main]: relative optimality gap < 0.001
0.0004555594215569907
2
0.14800000000000002
